## TASK 2: transfer from image classification

Image recognition is a machine learning area with several publications and data. The image recognition models are trained on much larger datasets than the music education systems~\cite{he2016deep}. The strategy is to convert the audio into a Mel-spectrogram and process it as a grey-scale image. The image recognition neural networks are very good at recognising patterns in their first layers. We want our feature extractor to take advantage of these patterns recognition in the first layers. Finally, the last layer of the classifier is trained on the good sounds data. In this work, we transfer learning from three image recognition state-of the-art-models, resnet18~\cite{he2016deep}, VGG19~\cite{simonyan2014very} and densenet161~\cite{huang2017densely}, to sound quality assessment.

In [1]:
! pip install -U pip
! pip install -U torch==1.5.1
! pip install -U torchaudio==0.5.1
! pip install -U torchvision==0.6.1
! pip install -U matplotlib==3.2.1
! pip install -U clearml>=0.16.1
! pip install -U pandas==1.0.4
! pip install -U numpy==1.18.4
! pip install -U tensorboard==2.2.1
!pip install git+https://github.com/mir-dataset-loaders/mirdata.git@Pedro/good_sounds

     |████████████████████████████████| 1.5MB 5.5MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 753.2 MB 14 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.8.0+cu101
    Uninstalling torch-1.8.0+cu101:
      Successfully uninstalled torch-1.8.0+cu101
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.9.0+cu101 requires torch==1.8.0, but you have torch 1.5.1 which is incompatible.
torchtext 0.9.0 requires torch==1.8.0, but you have torch 1.5.1 which is incompatible.
     |████████████████████████████████| 3.2 MB 4.6 MB/s 
     |████████████████████████████████| 6.6 MB 4.2 MB/s 
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.9.0+cu101
    Uninstalling torchvision-0.9.0+cu101:
  

     |████████████████████████████████| 10.1 MB 4.7 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.0.4 which is incompatible.


     |████████████████████████████████| 20.2 MB 43.2 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.9.0 requires torch==1.8.0, but you have torch 1.5.1 which is incompatible.
tensorflow 2.4.1 requires numpy~=1.19.2, but you have numpy 1.18.4 which is incompatible.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.0.4 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


     |████████████████████████████████| 3.0 MB 4.6 MB/s 
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.4.1 requires numpy~=1.19.2, but you have numpy 1.18.4 which is incompatible.
tensorflow 2.4.1 requires tensorboard~=2.4, but you have tensorboard 2.2.1 which is incompatible.
  Cloning https://github.com/mir-dataset-loaders/mirdata.git (to revision Pedro/good_sounds) to /tmp/pip-req-build-d8bzem56
  Running command git clone -q https://github.com/mir-dataset-loaders/mirdata.git /tmp/pip-req-build-d8bzem56
  Running command git checkout -b Pedro/good_sounds --track origin/Pedro/good_sounds
  Switched to a new branch 'Pedro/good_sounds'
  Branch 'Pedro/good_sounds' set up to tra

In [ ]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [ ]:
import PIL
import io

import pandas as pd
import numpy as np
from pathlib2 import Path
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter

import torchaudio
from torchvision.transforms import ToTensor
from torchvision import models

%matplotlib inline

In [ ]:

configuration_dict = {'number_of_epochs': 20, 'batch_size': 8, 'dropout': 0.3, 'base_lr': 0.005, 
                      'number_of_mel_filters': 64, 'resample_freq': 22050}


In [ ]:
import mirdata

from google.colab import drive
drive.mount('/content/drive')

g = mirdata.initialize("good_sounds", data_home="drive/MyDrive/good_sounds")

Mounted at /content/drive


## Data loader

In [ ]:
import sklearn 
import random

def get_data(dataset, klasses):
    # import pdb; pdb.set_trace()
    sound_selected = []
    track_ids, tracks = [], []
    if klasses == "all":
        for k, t in dataset.load_tracks().items():
            if t.get_sound_info['klass'] and t.get_sound_info['id'] not in sound_selected:
                track_ids.append(k)
                tracks.append(t)
                sound_selected.append(t.get_sound_info['id'])
    else:
        for klass in klasses:
          for k, t in dataset.load_tracks().items():
              if t.get_sound_info['instrument'] == klass and t.get_sound_info['klass'] and t.get_sound_info['id'] not in sound_selected:
                  track_ids.append(k)
                  tracks.append(t)
                  sound_selected.append(t.get_sound_info['id'])
    return track_ids, tracks


class good_soundsDataset(torch.utils.data.Dataset):
    def __init__(
        self,
        mirdataset,
        seq_duration=0.5,
        random_start=True,
        resample=8000,
        subset=0,
        train_split=0.8,
        test_split=0.2,
        random_seed=42,
        klasses=["violin"]
    ):
        """
        """
        self.klasses = klasses # a 'klass' or 'all' or 'family'
        self.seq_duration = seq_duration
        self.dataset = mirdataset
        track_ids, tracks = get_data(self.dataset, self.klasses)
        self.track_ids = track_ids
        self.tracks = tracks
        self.resample = resample
        self.set = subset
        self.random_start = random_start

        #### build a list with labels
        self.labels = {label:i for i,label in enumerate(['good', 'bad'])}
        full_labels = [x.get_sound_info['klass'] for x in tracks]
        #### build the three subsets: train, validation, test using train_test_split, a stratified split with the labels
        self.trackids_train, self.trackids_test = sklearn.model_selection.train_test_split(self.track_ids, train_size=1-test_split, random_state=random_seed, stratify=full_labels)
        train_labels = [l for l,i in zip(full_labels, self.track_ids) if i in self.trackids_train]
        self.trackids_train, self.trackids_valid = sklearn.model_selection.train_test_split(self.trackids_train, train_size=train_split, random_state=random_seed, stratify=train_labels)


    def __getitem__(self, index):

        #### get the file with index in the corresponding subset
        if self.set==0:
            track_id = self.trackids_train[index]
        elif self.set==1:
            track_id = self.trackids_valid[index]
        elif self.set==2:
            track_id = self.trackids_test[index]
        track = self.dataset.track(track_id)

        #### compute start and end frames to read from the disk
        si, ei = torchaudio.info(track.audio_path)
        sample_rate, channels, length = si.rate, si.channels, si.length
        duration = length / sample_rate
        if self.seq_duration>duration:
            offset = 0
            num_frames = length
        else:
            if self.random_start:
                #### we skip a number of seconds at the beginning of file 
                start = random.uniform(0, duration - self.seq_duration)
            else:
                start = 0.
            #### seconds to audio frames
            offset = int(np.floor(start * sample_rate))
            num_frames = int(np.floor(self.seq_duration * sample_rate))


        #### get audio frames corresponding to offset and num_frames from the disk
        audio_signal, sample_rate = torchaudio.load(filepath=track.audio_path, offset=offset,num_frames=num_frames)

        #### zero pad if the size is smaller than seq_duration
        seq_duration_samples = int(self.seq_duration * sample_rate)
        total_samples = audio_signal.shape[-1]
        if seq_duration_samples>total_samples:
            audio_signal = torch.nn.ConstantPad2d((0,seq_duration_samples-total_samples,0,0),0)(audio_signal)

        #### resample
        audio_signal = torchaudio.transforms.Resample(sample_rate, self.resample)(audio_signal)

        # This will convert audio files with two channels into one
        audio_signal = torch.mean(audio_signal, dim=0, keepdim=True)
               
        # Convert audio to log-scale Mel spectrogram
        melspectrogram_transform = torchaudio.transforms.MelSpectrogram(sample_rate=self.resample, n_mels=64)
        melspectrogram = melspectrogram_transform(audio_signal)
        melspectogram_db = torchaudio.transforms.AmplitudeToDB()(melspectrogram)
        
        #Make sure all spectrograms are the same size
        fixed_length = 3 * (self.resample//200)
        if melspectogram_db.shape[2] < fixed_length:
            melspectogram_db = torch.nn.functional.pad(melspectogram_db, (0, fixed_length - melspectogram_db.shape[2]))
        else:
            melspectogram_db = melspectogram_db[:, :, :fixed_length]

        # if self.audio[0]:
        #     fixed_length = 3 * self.resample
        #     if audio_signal.numel() < fixed_length:
        #         audio_signal = torch.nn.functional.pad(audio_signal, (0, fixed_length - audio_signal.numel())).numpy()
        #     else:
        #         audio_signal = audio_signal[0,:fixed_length].reshape(1,fixed_length).numpy()
        # else:
        #     audio_signal = np.array([])
        audio_signal = np.array([])

        return audio_signal, self.resample, torch.cat([melspectogram_db, melspectogram_db, melspectogram_db], dim=0), self.labels['good' if track.get_sound_info['klass'].startswith('good') else 'bad']

    def __len__(self):
        if self.set==0:
            return len(self.trackids_train)
        elif self.set==1:
            return len(self.trackids_valid)
        else:
            return len(self.trackids_test)

random_seed=0

train_dataset = good_soundsDataset(mirdataset=g, subset=0, random_seed=random_seed)
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=64,num_workers=4,pin_memory=True)
valid_dataset = good_soundsDataset(mirdataset=g, subset=1, random_seed=random_seed)
valid_loader = torch.utils.data.DataLoader(valid_dataset,batch_size=64,num_workers=4,pin_memory=True)
test_dataset = good_soundsDataset(mirdataset=g, subset=2, random_seed=random_seed)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=64,num_workers=4,pin_memory=True)

classes = ('good', 'bad')


In [ ]:
"kk"

'kk'

## Transfer learning from resnet18

In [ ]:
model = models.resnet18(pretrained=True)
# model.conv1 = nn.Conv2d(1, model.conv1.out_channels, kernel_size=model.conv1.kernel_size[0], 
#                       stride=model.conv1.stride[0], padding=model.conv1.padding[0])
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(*[nn.Dropout(p=configuration_dict.get('dropout', 0.25)), nn.Linear(num_ftrs, len(classes))])

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth


In [ ]:
optimizer = optim.SGD(model.parameters(), lr = configuration_dict.get('base_lr', 0.001), momentum = 0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = configuration_dict.get('number_of_epochs')//3, gamma = 0.1)
criterion = nn.CrossEntropyLoss()

In [ ]:
device = torch.cuda.current_device() if torch.cuda.is_available() else torch.device('cpu')
print('Device to use: {}'.format(device))
model.to(device)

Device to use: 0


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
tensorboard_writer = SummaryWriter('./tensorboard_logs')

In [ ]:
def plot_signal(signal, title, cmap=None):
    fig = plt.figure()
    if signal.ndim == 1:
        plt.plot(signal)
    else:
        plt.imshow(signal, cmap=cmap)    
    plt.title(title)
    
    plot_buf = io.BytesIO()
    plt.savefig(plot_buf, format='jpeg')
    plot_buf.seek(0)
    plt.close(fig)
    return ToTensor()(PIL.Image.open(plot_buf))

In [ ]:
def train(model, epoch):
    model.train()
    for batch_idx, (sounds, sample_rate, inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        iteration = epoch * len(train_loader) + batch_idx
        if batch_idx % log_interval == 0: #print training stats
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'
                  .format(epoch, batch_idx * len(inputs), len(train_loader.dataset), 
                          100. * batch_idx / len(train_loader), loss))
            tensorboard_writer.add_scalar('training loss/loss', loss, iteration)
            tensorboard_writer.add_scalar('learning rate/lr', optimizer.param_groups[0]['lr'], iteration)
                
        
        if batch_idx % debug_interval == 0:    # report debug image every "debug_interval" mini-batches
            for n, (inp, pred, label) in enumerate(zip(inputs, predicted, labels)):
                series = 'label_{}_pred_{}'.format(classes[label.cpu()], classes[pred.cpu()])
                # tensorboard_writer.add_image('Train MelSpectrogram samples/{}_{}_{}'.format(batch_idx, n, series), 
                #                              plot_signal(inp.cpu().numpy().squeeze(), series, 'hot'), iteration)

In [ ]:
import sklearn

def test(model, epoch):
    model.eval()
    class_correct = list(0. for i in range(2))
    class_total = list(0. for i in range(2))
    true = []
    predict = []
    with torch.no_grad():
        for idx, (sounds, sample_rate, inputs, labels) in enumerate(test_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)

            _, predicted = torch.max(outputs, 1)
            true = true + labels.tolist()
            predict = predict + predicted.tolist()
            c = (predicted == labels)
            for i in range(len(inputs)):
                label = labels[i].item()
                class_correct[label] += c[i].item()
                class_total[label] += 1
        
            iteration = (epoch + 1) * len(train_loader)

    total_accuracy = 100 * sum(class_correct)/sum(class_total)
    balanced_accuracy = sklearn.metrics.balanced_accuracy_score(true, predict)
    f_score = sklearn.metrics.f1_score(true, predict)
    precision = sklearn.metrics.precision_score(true, predict)
    recall = sklearn.metrics.recall_score(true, predict)
    cm = sklearn.metrics.confusion_matrix(true, predict)
    print('[Iteration {}] Accuracy on the {} test audios: {}%\n'.format(epoch, sum(class_total), total_accuracy))
    print('[Iteration {}] balanced accuracy on the {} test audios: {}%\n'.format(epoch, sum(class_total), balanced_accuracy))
    print('[Iteration {}] f_score on the {} test audios: {}%\n'.format(epoch, sum(class_total), f_score))
    print('[Iteration {}] precision on the {} test audios: {}%\n'.format(epoch, sum(class_total), precision))
    print('[Iteration {}] recall on the {} test audios: {}%\n'.format(epoch, sum(class_total), recall))
    print('[Iteration {}] cm on the {} test audios: {}%\n'.format(epoch, sum(class_total), cm))


In [ ]:
log_interval = 10
debug_interval = 25
for epoch in range(configuration_dict.get('number_of_epochs', 10)):
    print(epoch)
    train(model, epoch)
    test(model, epoch)
    scheduler.step()

0
Train Epoch: 0 [0/884 (0%)]	Loss: 0.493354
Train Epoch: 0 [640/884 (71%)]	Loss: 0.480135
[Iteration 0] Accuracy on the 277.0 test audios: 84.47653429602889%

[Iteration 0] balanced accuracy on the 277.0 test audios: 0.5%

[Iteration 0] f_score on the 277.0 test audios: 0.9158512720156556%

[Iteration 0] precision on the 277.0 test audios: 0.8447653429602888%

[Iteration 0] recall on the 277.0 test audios: 1.0%

[Iteration 0] cm on the 277.0 test audios: [[  0  43]
 [  0 234]]%

1
Train Epoch: 1 [0/884 (0%)]	Loss: 0.497629
Train Epoch: 1 [640/884 (71%)]	Loss: 0.386270
[Iteration 1] Accuracy on the 277.0 test audios: 84.47653429602889%

[Iteration 1] balanced accuracy on the 277.0 test audios: 0.5%

[Iteration 1] f_score on the 277.0 test audios: 0.9158512720156556%

[Iteration 1] precision on the 277.0 test audios: 0.8447653429602888%

[Iteration 1] recall on the 277.0 test audios: 1.0%

[Iteration 1] cm on the 277.0 test audios: [[  0  43]
 [  0 234]]%

2
Train Epoch: 2 [0/884 (0%)]	

## Transfer learning from resnet18

In [ ]:

model = models.resnet18(pretrained=True)
print(model)
num_ftrs = model.fc.in_features
for param in model.parameters():
    param.requires_grad = False
model.fc = nn.Sequential(*[nn.Dropout(p=configuration_dict.get('dropout', 0.25)), nn.Linear(num_ftrs, len(classes))])

optimizer = optim.SGD(model.parameters(), lr = configuration_dict.get('base_lr', 0.001), momentum = 0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = configuration_dict.get('number_of_epochs')//3, gamma = 0.1)
criterion = nn.CrossEntropyLoss()

device = torch.cuda.current_device() if torch.cuda.is_available() else torch.device('cpu')
print('Device to use: {}'.format(device))
model.to(device)
print("")

log_interval = 10
debug_interval = 25
for epoch in range(configuration_dict.get('number_of_epochs', 10)):
    print(epoch)
    train(model, epoch)
    test(model, epoch)
    scheduler.step()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## Transfer learning from densenet161

In [ ]:

model = models.densenet161(pretrained=True)
print(model)
num_ftrs = model.classifier.in_features
for param in model.parameters():
    param.requires_grad = False
model.classifier = nn.Sequential(*[nn.Dropout(p=configuration_dict.get('dropout', 0.25)), nn.Linear(num_ftrs, len(classes))])

optimizer = optim.SGD(model.parameters(), lr = configuration_dict.get('base_lr', 0.001), momentum = 0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = configuration_dict.get('number_of_epochs')//3, gamma = 0.1)
criterion = nn.CrossEntropyLoss()

device = torch.cuda.current_device() if torch.cuda.is_available() else torch.device('cpu')
print('Device to use: {}'.format(device))
model.to(device)
print("")

log_interval = 10
debug_interval = 25
for epoch in range(configuration_dict.get('number_of_epochs', 10)):
    print(epoch)
    train(model, epoch)
    test(model, epoch)
    scheduler.step()

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (rel

## Transfer learning from vgg19

In [ ]:

model = models.vgg19(pretrained=True)
print(model)
num_ftrs = model.classifier[6].in_features
for param in model.parameters():
    param.requires_grad = False
model.classifier[6] = nn.Sequential(*[nn.Dropout(p=configuration_dict.get('dropout', 0.25)), nn.Linear(num_ftrs, len(classes))])

optimizer = optim.SGD(model.parameters(), lr = configuration_dict.get('base_lr', 0.001), momentum = 0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = configuration_dict.get('number_of_epochs')//3, gamma = 0.1)
criterion = nn.CrossEntropyLoss()

device = torch.cuda.current_device() if torch.cuda.is_available() else torch.device('cpu')
print('Device to use: {}'.format(device))
model.to(device)
print("")

log_interval = 10
debug_interval = 25
for epoch in range(configuration_dict.get('number_of_epochs', 10)):
    print(epoch)
    train(model, epoch)
    test(model, epoch)
    scheduler.step()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd